Overleaf doc: https://www.overleaf.com/read/ysgbfhgmndrs

This notebook requires:
- jaxlib (`pip install jaxlib`)
- jax
- sympy
- scipy
- sympy2jax (`pip install sympy2jax`) -- but sympy's built-in lambdify should be sufficient for our uses

Not needed but in case we want to parse LaTeX:

- antlr4 python runtime (`conda install -c conda-forge antlr-python-runtime`) — for sympy's LaTeX parsing

# Setup

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import jax
import jax.numpy as jnp
import sympy
# from sympy.parsing.latex import parse_latex
# from sympy2jax import sympy2jax
import matplotlib.pyplot as plt
import scipy
import scipy.constants as const
import numpy as np
import math
import scipy.special as sp
from jaxopt import Bisection

In [3]:
from jax.config import config
config.update("jax_enable_x64", True)

In [4]:
e = np.e
ln = np.log
ln_j = jnp.log
gamma_0 = sp.exp1
gamma_0_j = jax.scipy.special.exp1

In [5]:
%matplotlib notebook

In [311]:
def plug_in_values(equation, values_in):
    jax_equation = sympy.lambdify(list(equation.free_symbols), equation, jnp)
    result = jax_equation(*tuple(values_in[_.name] for _ in list(equation.free_symbols)))
    return result

---

# Units used

* Temperature: keV
* Density: cm^-3
* Field: T

# Cary's spreadsheet sanity check

Spreadsheet benchmark inputs:
* $B_p = 6$ T
* $B_{p,m} = 30$ T
*     --> $R_p = 5$
* $\beta_{limit} = 0.8$
* $E_b = 1000$ keV
* $r_b = 0.25$ m
* $L_p = 4$ m

Yields (ignoring finite $\beta$ effects on the electron temperature):
* $T_e = 77.1$ keV
* $n_{plug} = 9.68 \times 10^{19}$ m$^{-3}$

# Fundamental parameters

I'm filling out the parameters as I go along

## Sympy variables

In [312]:
# Machine / engineering parameters
[Eb, # keV
 beta,
 B_pm, # Tesla -- plug 
 B_p, # Tesla
 r_b,
 L_p,
 Ti,  # Placeholder (will be calculated later), keV
 Te,  # Placeholder (will be calculated later), keV
] = sympy.symbols('Eb '
                  'beta '
                  'B_pm '
                  'B_p '
                  'r_b '
                  'L_p '
                  'Ti '
                  'Te ')

# Physics parameters?
[mu,
 Z_eff,
 I_cooling,
 tau_alpha
] = sympy.symbols('mu '
                  'Z_eff '
                  'I_cooling '
                  'tau_alpha')

## JAX variables to optimize for

In [313]:
values = {
    'Eb': 1000,
    'beta': 0.8,
    'B_pm': 30,
    'B_p': 6,
    'r_b': 0.25,
    'L_p': 4,
    
    'mu': 2.5,
    'Z_eff': 1,
    'I_cooling': 0,
    'tau_alpha': 0
}

# Constants

In [314]:
E_alpha = 3.5  # MeV

# General formulae

## Cross section / reactivity paramterization

Accepts: ion temperature in keV. Yields: reactivity in cm$^3$/s

These constants are valid for 0.2-100 keV (Maxwellian) ion temperatures. Verified correct to 4 decimal places for 0.2, 1, and 10 keV.

In [315]:
DT_BG = 34.3827 # keV^0.5
DT_mrc2 = 1124656 # keV
DT_C1 = 1.17302e-9
DT_C2 = 1.51361e-2
DT_C3 = 7.51886e-2
DT_C4 = 4.60643e-3
DT_C5 = 1.35e-2
DT_C6 = -1.06750e-4
DT_C7 = 1.366e-5

In [316]:
DD_pT_BG = 31.3970 # keV^0.5
DD_pT_mrc2 = 937814 # keV
DD_pT_C1 = 5.65718e-12
DD_pT_C2 = 3.41267e-3
DD_pT_C3 = 1.99167e-3
DD_pT_C4 = 0
DD_pT_C5 = 1.05060e-5
DD_pT_C6 = 0
DD_pT_C7 = 0

In [317]:
DD_nHe_BG = 31.3970 # keV^0.5
DD_nHe_mrc2 = 937814 # keV
DD_nHe_C1 = 5.43360e-12
DD_nHe_C2 = 5.85778e-3
DD_nHe_C3 = 7.68222e-3
DD_nHe_C4 = 0
DD_nHe_C5 = -2.96400e-6
DD_nHe_C6 = 0
DD_nHe_C7 = 0

In [318]:
DT_theta = Ti / (1 -  Ti * (DT_C2 + Ti * (DT_C4 + Ti * DT_C6)) / (1 + Ti * (DT_C3 + Ti * (DT_C5 + Ti * DT_C7))))
DT_xi = (DT_BG ** 2 / (4 * DT_theta)) ** (1/3)
DT_reactivity = DT_C1 * DT_theta * (DT_xi / (DT_mrc2 * Ti ** 3)) ** 0.5 * sympy.E ** (-3 * DT_xi)

In [319]:
DD_pT_theta = Ti / (1 -  Ti * (DD_pT_C2 + Ti * (DD_pT_C4 + Ti * DD_pT_C6)) / (1 + Ti * (DD_pT_C3 + Ti * (DD_pT_C5 + Ti * DD_pT_C7))))
DD_pT_xi = (DD_pT_BG ** 2 / (4 * DD_pT_theta)) ** (1/3)
DD_pT_reactivity = DD_pT_C1 * DD_pT_theta * (DD_pT_xi / (DD_pT_mrc2 * Ti ** 3)) ** 0.5 * sympy.E ** (-3 * DD_pT_xi)

In [320]:
DD_nHe_theta = Ti / (1 -  Ti * (DD_nHe_C2 + Ti * (DD_nHe_C4 + Ti * DD_nHe_C6)) / (1 + Ti * (DD_nHe_C3 + Ti * (DD_nHe_C5 + Ti * DD_nHe_C7))))
DD_nHe_xi = (DD_nHe_BG ** 2 / (4 * DD_nHe_theta)) ** (1/3)
DD_nHe_reactivity = DD_nHe_C1 * DD_nHe_theta * (DD_nHe_xi / (DD_nHe_mrc2 * Ti ** 3)) ** 0.5 * sympy.E ** (-3 * DD_nHe_xi)

In [321]:
jax_DT_reactivity = sympy.lambdify(list(DT_reactivity.free_symbols), DT_reactivity, jax.numpy)
jax_DD_pT_reactivity = sympy.lambdify(list(DD_pT_reactivity.free_symbols), DD_pT_reactivity, jax.numpy)
jax_DD_nHe_reactivity = sympy.lambdify(list(DD_nHe_reactivity.free_symbols), DD_nHe_reactivity, jax.numpy)

In [322]:
jax_DD_nHe_reactivity(10 * 3/2)

DeviceArray(1.48101044e-18, dtype=float64, weak_type=True)

# Derived quantities

Mirror ratio $R_{\text{plug}} = \frac{B_m}{B_p}$

In [323]:
R_p = B_pm / B_p; R_p

B_pm/B_p

## Solving for Ti and Te

Ion temperature: $\frac{3}{2} \frac{T_i}{E_{beam}} = \frac{\exp{(-\alpha)} - \alpha \Gamma(0, \alpha)}{\Gamma(0, \alpha)}$

Electron temperature: $\frac{T_e}{E_{beam}} =  \left( \frac{T_i}{E_{beam}} \frac{2}{3} \frac{\alpha^2 \ln{R_m}^2}{(22.4)^2} \right)^{1/3}$

Energy balance: $E_{beam} + p_{aux} = T_i + 6T_e$

In [324]:
def alpha_root_func(a, R):
    return ((e ** (-a) / gamma_0_j(a) - a))*2/3 + 6 * (((e ** (-a) / gamma_0_j(a) - a)) * a ** 2 * ln_j(R) ** 2/(22.4) ** 2 * 3/2) ** (1/3)

def alpha_eq(a, R_in, power_aux_in):
    return alpha_root_func(a, R_in) - (1 + power_aux_in)

def find_alpha(R_in, power_aux_in):
    bisec = Bisection(optimality_fun=alpha_eq, lower=0.001, upper=200.0, tol=1e-3, check_bracket=True)
    alpha = bisec.run(R_in=R_in, power_aux_in=power_aux_in).params
    return alpha

def Ti_given_alpha(a):
    return ((e ** (-a) / gamma_0_j(a) - a))*2/3

def Te_given_alpha(a, R):
    return (((e ** (-a) / gamma_0_j(a) - a)) * a ** 2 * ln_j(R) ** 2/(22.4) ** 2 * 2/3) ** (1/3)

def find_Ti_and_Te(values_in, power_aux_in):
    R_in = values_in['B_pm'] / values_in['B_p']
    E_beam = values_in['Eb']
    
    alpha = find_alpha(R_in, power_aux_in)
    Ti = Ti_given_alpha(alpha)
    Te = Te_given_alpha(alpha, R_in)
    return Ti * E_beam, Te * E_beam

### Te (old)

Electron temperature 
$T_e = 0.089 E_b \log_{10} \left( R_p \right)^{0.4} = 0.089 E_b \log_{10} \left(\frac{R_p}{1-\beta} \right)^{0.4}$

In [325]:
# Te = 0.089 * Eb * ((sympy.log(R_p / (1))/sympy.log(10)) ** 0.4); Te

In [326]:
# plug_in_values(Te, values)

### Ti (old)

In [327]:
# Ti = 0.6 * Eb

## End cells / plugs

Radius at midplane $a_{\text{plug}} = r_b \sqrt{\frac{B_m}{B_p}} = r_b \sqrt{R_{plug}}$

In [328]:
a_plug = r_b * sympy.sqrt(R_p); a_plug

r_b*sqrt(B_pm/B_p)

Volume $V_p = L_p \pi a_p^2$

In [329]:
V_p = L_p * sympy.pi * a_plug ** 2; V_p

pi*B_pm*L_p*r_b**2/B_p

### n_plug

Density at a given beta: $n_{\text{plug}} = B_p^2 \frac{\beta_{\text{limit}}}{2\mu_0 |e|\left(T_{\text{ion}} + T_e \right)}$

In [330]:
const.mu_0 * const.elementary_charge * 2 * 1000 / 1e-20

0.04026709076694141

In [331]:
n_plug = (B_p) ** 2 * beta / (2 * const.mu_0 * const.elementary_charge * 1000 * (Ti + Te))

In [332]:
plug_in_values(n_plug, values)

KeyError: 'Ti'

Total particle number: $N_{\text{tot}} = V_p n_{\text{plug}}$

In [ ]:
N_tot = V_p * n_plug; N_tot

### tau fowler/baldwin

**General formula:** Particle conefinement time $\tau_{\text{Fowler/Baldwin}} = 2.8 \cdot 10^{12} \frac{E_b^{3/2}}{n_e} \log_{10} R_m$

In [ ]:
tau_Fowler_Baldwin = 1.4 * 10 ** 16 * Eb ** (3/2) / n_plug * sympy.log(R_p / sympy.sqrt(1 - beta)) / sympy.log(10); tau_Fowler_Baldwin

In [ ]:
plug_in_values(tau_Fowler_Baldwin, values)

Particles lost per second: $\frac{dN}{dt} = \frac{N_{\text{tot}}}{\tau_{\text{Fowler/Baldwin}}}$

In [ ]:
dN_dt = N_tot / tau_Fowler_Baldwin

**General formula:** Ion gyroradius at center of the plug: $\rho_i = \frac{m v_\perp}{q B} = 3.22\cdot 10^{-3} \frac{\sqrt{\mu E_{\text{ion}}}}{B_p}$

In [ ]:
rho_i = 3.22 * 10 ** -3 * sympy.sqrt(mu * Ti) / B_p; rho_i

Number of gryoradii in the plasma radius: $N_{\text{gyro}} = \frac{a_p}{\rho_i}$

In [ ]:
N_gyro = a_plug / rho_i; N_gyro

NBI current (A): $I_{\text{NBI}}=|e|\frac{dN}{dt}$

In [ ]:
I_NBI = const.elementary_charge * dN_dt

**General formula:** Coulomb logarithms: $\lambda_{ei} = 24 - 0.5\ln{n_e} + \ln{T_e}$

In [ ]:
# lambda_ei = 24 - 0.5 * sympy.log(n_plug) + sympy.log(Te)

**General formula:** Slowing down times: $\tau_{i,\text{slow}} = 0.1 \frac{\mu T_e^{3/2}}{n_{20} Z^2 \lambda_{ei}}$

In [ ]:
tau_i_slow = 0.1 * mu * Te ** (3/2) / (n_plug / 10 ** 20 * Z_eff ** 2 * lambda_ei)

Electron heating by fast ions (MW): $10^{-3}\frac{I_{\text{NBI}}E_b}{\tau_{\text{slow}}}$

In [ ]:
P_e_heating_fastI = 10 ** -3 * I_NBI * Eb / tau_i_slow

**General formula:** Lorentz factor: $\gamma = \sqrt{1 - \frac{T_e}{m_e c^2}} = \sqrt{1 - \frac{T_e}{511 \text{keV}}}$

In [ ]:
gamma = sympy.sqrt(1 - Te / 511); gamma

Synchrotron radiation power loss (MW): $ P_{\text{synch}} = 6\cdot10^{-3} V_p n_{20} T_e \gamma^2 B_p^2$

In [ ]:
P_synch = 6 * 10 ** -3 * V_p * n_plug / 10 ** 20 * Te * gamma ** 2 * B_p ** 2; P_synch

Bremsstrahlung radiation power loss (MW): $P_{\text{brem}} = 5.35\cdot10^{-3} n_{20}^2 Z_{\text{eff}}\sqrt{T_e} V_p$

In [ ]:
P_brem = 5.35 * 10 ** -3 * (n_plug / 10 ** 20) ** 2 * Z_eff * sympy.sqrt(Te) * V_p

Power loss from escaping electrons (MW): $P_{\text{e,endloss}} = 10^{-3}(I_{\text{NBI}}+I_{\text{cooling}})\cdot 7 T_e$

In [ ]:
P_e_endloss = 10 ** -3 * (I_NBI + I_cooling) * 7 * Te; P_e_endloss

Power loss from escaping fast ions (MW): $P_{\text{i,endloss}} = 10^{-3} I_{\text{NBI}} \left(E_b-T_e\right)$

In [ ]:
P_i_endloss = 10 ** -3 * I_NBI * (Eb - Te); P_i_endloss

Injected NBI Power (MW): $P_{\text{NBI}} = 10^{-3}I_{\text{NBI}} E_b$

In [ ]:
P_NBI = 10 ** -3 * I_NBI * Eb; P_NBI

In [ ]:
plug_in_values(P_NBI, values)

Injected ECH Power (MW): $P_{\text{ECH}} = \frac{P_{\text{synch}}}{20} +P_{\text{e,endloss}} - \left(\text{Electron heating from fast ions}\right)$

In [ ]:
P_ECH = P_synch / 20 #+ P_e_endloss - P_e_heating_fastI

DT fusion reaction rate (#/s). Here we assume a 50-50 DT fuel mixture: $R_{\text{x,plug,DT}}=V_p \frac{n_{\text{plug}}^2}{4} \langle\sigma v\rangle_{DT}$

In [ ]:
Rx_plug_DT = V_p * n_plug ** 2 / 4 * DT_reactivity

DD fusion reaction rate (#/s). Here we assume a 50-50 DT fuel mixture: $R_{\text{x,plug,DD}}=V_p n_{\text{plug}}^2 \langle\sigma v\rangle_{DD}$ (I think this should have a 1/2 someplace)

In [ ]:
Rx_plug_DD = V_p * n_plug ** 2 * (DD_pT_reactivity + DD_nHe_reactivity) / 2

Fusion power (MW): $P_{\text{plug}}=17.6|e|R_{\text{x,plug}}$ -- we're just focusing on DT for now. Including DD it'd be: $P_{\text{plug}}=17.6|e|R_{\text{x,plug,DT}} + \frac{4.03+3.27}{2}|e|R_{\text{x,plug,DD}}$

In [ ]:
P_fus_plug = 17.6 * const.elementary_charge * Rx_plug_DT * 1e-6  # 1e-6 converts W to MW

Lawson Triple Product (keV$\cdot$s/m$^3$): $\tau_{\text{Fowler/Baldwin}} n T_i$

In [ ]:
triple_product = tau_Fowler_Baldwin * n_plug * Ti

Neutron Flux, DT only (MW/m$^2$): $\frac{14}{17.6}\frac{P_{\text{plug}}}{4\pi a_{\text{wall}}^2}$

In [ ]:
neutron_flux = 14 / 17.6 * P_fus_plug / (4 * const.pi * a_plug ** 2)

Burnup fraction, DT: $\frac{R_{\text{x,plug,DT}}}{dN/dt}$

In [ ]:
frac_burnup = Rx_plug_DT / dN_dt

# Q_plug

## Evaluate Ti and Te

$Q_{\text{plug}}$: $Q_{\text{plug}} = \frac{P_{\text{plug}}}{P_{\text{injected}}}$ (power in MW)

In [ ]:
plug_in_values(P_ECH, values)

In [ ]:
Q_plug = P_fus_plug / (P_NBI + P_ECH)

In [ ]:
def calc_Q(values, p_aux):
    Ti_Te_dict = {}
    Ti_Te_dict['Ti'], Ti_Te_dict['Te'] = find_Ti_and_Te(values, p_aux)
    Q = plug_in_values(Q_plug, {**values, **Ti_Te_dict})
    return Q

In [ ]:
temp_values = {**values, 'B_p': 6}; temp_values

In [ ]:
find_Ti_and_Te(values, 0)

In [ ]:
calc_Q(temp_values, 0)

* Ti/Ebeam = 2/3 --> Q_plug = 0.68327016 (spreadsheet says 0.686)
* Ti/Ebeam = 0.6 --> Q_plug = 0.7356443

### Triple product, energy confinement time, etc...

In [ ]:
plug_in_values(triple_product, values)

In [ ]:
plug_in_values(n_plug * tau_Fowler_Baldwin, values)

In [ ]:
plug_in_values(P_fus_plug, {**values, **Ti_Te_dict})

Energy confinement time $\tau_E = W_p / P_{\text{heating}}$ (power balance) (volume is overestimated — a linear falloff would cut it by a third):

In [ ]:
plug_in_values((V_p * n_plug * 1000 * const.elementary_charge * (Ti + Te) * 3/2) / (1e6 * P_NBI), values)

$\alpha$ particle density ($10^{20}m^{-3}$): $n_{\alpha} = \frac{I_{\text{NBI}} Q_{\text{plug}} \tau_{\alpha} E_b}{16 V_p E_{\alpha}}$

In [ ]:
n_alpha = I_NBI * Q_plug * tau_alpha * Eb / (16 * V_p * E_alpha)

$Z_{\text{eff}}$: $Z_{\text{eff}} = 1 +2n_{\alpha}$

In [ ]:
Z_eff = 1 + 2 * n_alpha